In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score

###**For Confusion Matrix**

In [ ]:
def confusion_matrix(y_test,Predicted_class):
  FP=0
  TP=0
  FN=0
  TN=0
  for j in range(len(y_test)):
    if Predicted_class[j]==0:
      if y_test[j]==0:
        TN+=1
      else:
        FN+=1
    else:
      if Predicted_class[j]==1:
        if y_test[j]==1:
          TP+=1
        else:
          FP+=1
    cm=np.array([[TN,FP],[FN,TP]])
  return cm

###**For Classwise Accuracy**

In [ ]:
def class_accuracy(y_test,model):
  mat = confusion_matrix(y_test,model)
  classacc = (((mat[0][0])/(mat[0][0]+mat[0][1]) + (mat[1][1])/(mat[1][1] + mat[1][0]))/2)*100
  return classacc

###**For Decision Boundary**

In [ ]:
def decbound(model,X,y,f1,f2):    # Made the function for decision boundary
    x_, y_ = np.meshgrid(np.linspace(np.min(X[f1]), np.max(X[f1]), 100), np.linspace(np.min(X[f2]), np.max(X[f2]), 100)) #Created meshgrid
    model.fit(X[[f1,f2]],y)          #Trained the model
    Z = model.predict(pd.DataFrame(np.c_[x_.ravel(), y_.ravel()],columns = [f1,f2]))  #Predicted using the model
    Z = Z.reshape(x_.shape)
    plt.contourf(x_, y_, Z, cmap=plt.cm.RdYlBu)      #Did the contour plotting
    sns.scatterplot(data=X[[f1,f2]],x = f1,y = f2,hue = y)   #Did the scatter plotting
    return plt.show()

###**For Storing Node of Decision Tree**

In [ ]:
class node:                                      #Creating an object to store data
    def __init__(self, X, y, feature):
        self.X = X
        self.y = y
        self.feature = feature
        self.childs = {}
        self.decision = None

    def isleaf(self):
      if len(set(y))==1:
        return 1
      else:
        return 0

###**For DecisionTreeClassifier**(Gini Index as criteria)

In [ ]:
class Dtreeclassifier:       # Made an object for DecisionTreeClasifier
    def __init__(self,X,y,max_depth=2):     #Made constructor to take inputs
        self.root = None
        self.X = X
        self.y = y
        self.max_depth = max_depth
        
    def gini(self,x):                      # For calculating gini index
      uni,freq = np.unique(x,return_counts=True)
      p = freq/len(x)
      totprobab = np.sum(p ** 2)
      self.ginindex = 1 - totprobab
      return self.ginindex

    def impurity(self,X,y,feature,splitter):     #For gini impurity cost which would be useful for finding best feature for split
      Xlside,ylside,Xrside,yrside=[],[],[],[]
      tot = len(X)
      for i in X.index:
        if X[feature][i] <= splitter:
          Xlside.append(X[feature][i])
          ylside.append(y[i])
        else:
          Xrside.append(X[feature][i])
          yrside.append(y[i])
      lside = len(Xlside)
      rside = len(Xrside)
      Xleft = pd.Series(Xlside,dtype = 'float64')
      Xright = pd.Series(Xrside,dtype = 'float64')
      yleft = pd.Series(ylside,dtype = 'float64')
      yright = pd.Series(yrside,dtype = 'float64')
      self.gini_impurity = (lside / tot)*self.gini(yleft) + (rside / tot)*self.gini(yright)  #It will also help in finding best split value
      return self.gini_impurity 

    def fbest(self,X, y):       #Made function for selecting best feature for split
      best = ''          #Initializing best feature as none
      bestimpurity = float('inf')   #Initializing bestimpurity as big value
      for i in X.columns:                    #Iterating over every feature
          score = self.impurity(X, y, i, 0)       #Finding impurity w.r.t to feature i and splitvalue 0
          if score < bestimpurity:
              bestimpurity = score           #If condition satisfied updating best impurity as that score
              best = i                       #Updating best feature value as one going on in loop
      return best

    def split(self,X,y,max_depth):   #For splitting the data recursively
      if max_depth == 0 or len(X) == 0:
          leaf = node(X,y,None)
          return leaf
      if gini(y) == 0:
          leaf = node(X,y,None)
          return leaf
      f = self.fbest(X,y)
      root = node(X,y,feature=f)
      left = X[f] == 0
      right = X[f] == 1  
      if f in ['island', 'year']:
          mid = X[f] == 2
          root.childs['middle'] = self.split(X[mid],y[mid], max_depth-1)
      if len(X[left]) > 0:
          root.childs['left'] = self.split(X[left],y[left], max_depth-1)
      if len(X[right]) > 0:
          root.childs['right'] = self.split(X[right],y[right], max_depth-1)
      return root 

    def train(self, X, y):                  #For Training the data
      self.root = self.split(X, y, self.max_depth)  

    def predict(self, x,s):                   #For prediction of data
      if s.feature == None:
        t = np.array(s.y)
        t,freq = np.unique(t,return_counts  =True)
        return t[np.argmax(freq)]
      
      column = []
      for i in s.X.columns:
        column += [i]
      
      index = column.index(s.feature)
      if x[index] == 0:
        return self.predict(x,s.childs['left'])
      elif x[index] == 1:
        return self.predict(x,s.childs['right'])
      else:
        return self.predict(x,s.childs['middle'])

    def test(self,X):
      o = np.array(X)
      y_pred = []
      for i in o:
        ans = self.predict(i,self.root)
        y_pred.append(ans)
      return y_pred
    
    def accuracy(self,y_test,y_pred):
      v = np.array(y_test)
      c=0
      for i in range(len(v)):
        if v[i] == y_pred[i]:
          c+=1
      return c/len(y_test)

###**For Continuos to Categorical**(Dataset specific)

In [ ]:
def cont_to_cat(X,y):       #Function to convert continuos data to categorical
  f = ['bill_depth_mm','bill_length_mm','flipper_length_mm','body_mass_g']
  split_store = []
  for I in f:
    n = (max(X[I])-min(X[I]))/2000     #Deciding the value to step in split values according to feature
    splt = np.arange(min(X[I]),max(X[I]),n)     #Making array of split values 
    gini=[]
    for i in splt:
      gini.append(impurity(X,y,I,i))         #Making list of gini_impuruty according to feature value
    s=splt[np.argmin(gini)]                   #Minimum gini will be best split so taking the index of that feature value
    split_store.append(s)                
    new = []                                 #Using that index finding the feature value for split
    for i in X.index:
      if X[I][i] <= s:
        new.append(0)
      else:
        new.append(1)
    X[I] = new
  return X,split_store                  #Returning categorized data

###**Example of ROC-AUC Curve**

In [ ]:
fpr,tpr,thresholds = roc_curve(y_test,y_pred)
roc_auc = roc_auc_score(y_test,y_pred)  #computing roc_auc score
print(roc_auc)

plt.plot(fpr, tpr, label='ROC curve')   #plotting roc curve
plt.plot([0, 1], [0, 1], 'k--')  # random guess line
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()

###**To discretize features**

In [ ]:
def discreter(data, bins):                     #Creating function to Discretize the features
  width = (max(data)+0.1 - min(data))/bins     #Computing Width u
  bin = []                                     #Making list of bin edges
  for i in range(bins+1):
    z = min(data) + i*width
    bin.append(z)                              #Appending in bin edges list
  k=[]
  for j in data:
    for i in range(bins+1):
      if(j>=bin[i] and j<bin[i+1]):          #Discretizing the continuos data using various conditional
        k.append(i+1)
        break
    else:
      if bin[bins+1]<=j:
        k.append(bins+1)
  return k

###**For Calculating Likelihood Probability**

In [ ]:
def likelihood(data):        #Creating function for calculating Likelihood/Conditional probability
  X = {}
  for i in data.columns:
    p = data[i].value_counts()    #counting unique values
    l = p.index                   #Making list of unique values
    d = {}                        #Creating Dictionary of Probability
    for j in l:
      prob = p[j]/len(data)
      d[j] = prob
    for k in range(1,26):
      if k in d:
        continue
      else:
        d[k] = 0
    X[i] = d
  return X  

###**For Calculating Posterior Probability**(Data Specific)

In [ ]:
def post(vertical,prior,bins):      #Made function to calculate posterior probability
  evidence = 0
  post1,post2,post3=[],[],[]
  #prior is same for all there i.e 0.3333.
  for j in range(1,bins+1):
    for i in range(1,4):
      evidence +=  likelihoods[i][vertical][j]*prior   #calculated evidence
    l1 = likelihoods[1][vertical][j]                   #conditional probability of class1
    l2 = likelihoods[2][vertical][j]                   #conditional probability of class2
    l3 = likelihoods[3][vertical][j]                   #conditional probability of class3
    p1 = (l1*prior)/evidence                           #Posterior probability of class1
    p2 = (l2*prior)/evidence                           #Posterior probability of class2
    p3 = (l3*prior)/evidence                           #Posterior probability of class3
    post1.append(p1)
    post2.append(p2)
    post3.append(p3)
  return post1,post2,post3

###**For GaussianNB Classifier**

In [ ]:
class GaussianBayesClassifier():
    def __init__(self, variant=1):
        self.variant = variant
    def train(self,X, y, variant=1):
        self.classes, class_prior = np.unique(y, return_counts=True)  #Computing class priors
        class_prior = class_prior / len(y)
        self.class_priors = {}
        k=0
        for i in self.classes:
          self.class_priors[i] = class_prior[k]
          k+=1
        self.means = {}                         # Computing class means and variances for each feature
        self.variances = {}
        for c in self.classes:
          X_c = X[y == c]
          self.means[c] = np.array(X_c.mean(axis=0))     
          self.variances[c] = np.cov(X_c.T)
          f = np.var(X,axis=0)[0]
          if self.variant == 2:                 # Applying changes in variances according to variant.
            self.variances[c] = np.cov(X.T)
          elif self.variant == 3:
            nf = len(X_c.columns)
            self.variances[c] = f*(np.identity(nf))
          

        return self.classes, self.class_priors, self.means, self.variances

    def test(self,X, classes, class_priors, means, variances):
      self.pred = []
      for i in X.index:
        x5 = np.array(X.loc[i,:])
        l = []
        for j in classes:
          probability = -0.5*(x5.T - means[j].T)@(np.linalg.inv(variances[j]))@(x5 - means[j]) - 0.5*np.log(np.linalg.det(variances[j])) + np.log(class_priors[j])
          l.append(probability)
        self.pred.append(classes[np.argmax(l)])
      self.pred = np.array(self.pred)
      return self.pred

    def score(self,y):
        cnt = 0
        p = list(y)
        for i in range(len(p)):
          if p[i] == self.pred[i]:
            cnt+=1
        return cnt/len(y)
    def split(self,w):
     train=self.w.sample(frac=0.7, replace=False, random_state=1)
     test=self.w.drop(train.index)
     a=train.shape[0]
     b=test.shape[0]

     return (a,b)
     
    def decbound(self,X,y,f1,f2):
      x_, y_ = np.meshgrid(np.linspace(np.min(X[f1]), np.max(X[f1]), 100), np.linspace(np.min(X[f2]), np.max(X[f2]), 100))
      self.classes,self.class_priors,self.means, self.variances = self.train(X[[f1,f2]],y,self.variant)
      Z = self.test(pd.DataFrame(np.c_[x_.ravel(), y_.ravel()],columns = [f1,f2]),self.classes,self.class_priors,self.means, self.variances)
      Z = Z.reshape(x_.shape)
      plt.contourf(x_, y_, Z, cmap=plt.cm.RdYlBu)
      sns.scatterplot(data=X[[f1,f2]],x = f1,y = f2,hue = y)
      plt.show()

###**For Bagging Classifier**

In [ ]:
class _Bagging_:
  def __init__(self,n_estimators,samples):              #Initializing Input Variables and making them global.
    self.n_estimators = n_estimators
    self.samples = samples

  def fit(self,X,y):                                   #Used for training the model of Training Datasets.
    self.X,self.y = X,y                                #Making Input variables global.
    self.x_t,self.y_t=[],[]
    self.models = []
    X = X.to_numpy()
    y = y.to_numpy()
    for i in range(self.n_estimators):                 #Iterates over n_estimators=10 
      indexes = np.random.choice(len(X),round(self.samples*len(X)),replace = True)  #Gives random index from training set.
      Xt,yt = X[indexes],y[indexes]                    #Assigns random samples for training
      self.x_t.append(Xt)
      self.y_t.append(yt)
      mod = dtc()                                      #mod = dtc() which is DecisionTreeClassifier() used for training
      a = mod.fit(Xt,yt)                               
      self.models.append(a)                            #Making a list of trained models. 
    self.x_t,self.y_t=pd.Series(self.x_t),pd.Series(self.y_t)

  def predict(self,X_test):                            #Used to Predict the output y_pred
    self.y_pred = []
    X_test = X_test.values
    for i in range(len(X_test)):
      pred = []
      for j in self.models:
        pred.append(j.predict(X_test[i].reshape(1,-1))[0])     #Predicting from each trained model.
      real_pred=max(set(pred), key = pred.count)               #Taking Majority of prediction which is basis of Voting Classifier.
      self.y_pred.append(real_pred)                            #Making list of outputs.

    return np.array(self.y_pred)                               #Returning array of outputs.

  def accuracies(self,X_test,y_test):                  #Used to give Accuracy of each tree for summarizing each tree performance numerically.
      self.Accuracy={}                               
      self.predicted = []
      y_new = np.array(y_test)
      for i,j in zip(range(1,self.n_estimators+1),self.models):
          y_p =j.predict(X_test)
          self.predicted.append(y_p)
          x=0
          for k in range(len(y_p)):
            if y_p[k] == y_new[k]:                    
              x += 1
          self.Accuracy[i]= (x/len(y_p))                    #Making Dictionary of the accuracies
      return self.Accuracy                                  #Returning the dictionary

  def visual(self):                                    #Used for Summarizing each tree performance visually
    plt.plot(self.Accuracy.keys(),self.Accuracy.values())
    plt.xlabel('Tree Number')
    plt.ylabel('Accuracy')
    plt.title('Visual Summarization of Each Tree')

###**For K-Means Clustering**

In [ ]:
class kmeans_cluster:
  def __init__(self,k,max_iters):            #Initializing variables to input, value of k to be given by user.
    self.k = k
    self.max_iters  = max_iters

  def fit(self,X,init_centroid):             #Used to predict the labels and update the centroid, taking initial cluster point from user.                                
    self.init_centroid = init_centroid
    n_samples,n_features = X.shape
    for i in range(self.max_iters):
      print                             #Stop iterating when max_iter is reached.
      self.centroids = np.zeros((self.k, n_features))
      self.labels = []
      for j in X:
        dist = np.sqrt(((init_centroid - j)**2).sum(axis=1))     #Computing distance
        self.labels.append(np.argmin(dist))                      #Setting label w.r.t minimum distance.
      self.labels = np.array(self.labels)
      for j in range(len(self.init_centroid)):
        if np.sum(self.labels == j) > 0:
          self.centroids[j] = np.mean(X[self.labels == j], axis=0)
      if (self.init_centroid == self.centroids).all() == True:        #Stop iterating when cluster centers not are changing.
        break
      self.init_centroid = self.centroids.copy()                      #Updating centroid

  def label(self):                                                    #Used to return label.
    a = np.array(self.labels)
    return a

  def centroid(self):                                                 #Used for returning the centroids.
    return self.centroids

  def SSE(self,X,centroid):                                           #Used to compute Sum of squared Error.
    sse = 0
    for i in range(X.shape[0]):
      sq_dist = []
      for j in range(self.k):
        b = np.sum((X[i]-centroid[j])**2)
        sq_dist.append(b)
      sse += min(sq_dist)
    return sse

###**For PCA**

In [ ]:
class pca:
  def __init__(self,n=None):                         #Constructor for initializng variables for input. 
    self.n = n                                       # n = No. of Components

  def covmat(self,X_):                                #For Computing the Co-variance Matrix             
    mean,std = [],[]
    for i in range(X_.shape[1]):                 
      mean.append(np.mean(X_.iloc[:,i],axis=0))       #Finding Mean of the column.
      std.append(np.std(X_.iloc[:,i],axis=0))         #Finding Std. Deviation of the column.
      a = (X_.iloc[:,i] - mean[i])
      b = a/std[i]
      X_.iloc[:,i] = b                                #Standardizing the DataFrame Columnwise.
      self.matrix = (X_.T@X_)/(X_.shape[0]-1)

    return self.matrix                               #Returning the Covariance Matrix.

  def fit(self,X_):
    eigvect = np.linalg.eig(self.matrix)[1]         #Computing EigenVectors using inbuilt
    lis = []
    eigenvalues,eigenvects = [],[]
    for i in range(eigvect.shape[1]):
        v = eigvect[:, i]
        eigval = np.dot(self.matrix,v) / v           #Finding EigenValues from scratch.
        eigvals = eigval[0]
        eigenvalues.append(eigvals)
        eigenvects.append(v)
        lis.append((eigvals,v))                      #Made a list of lists having eigenvalues and eigenvectors. 
    lis = sorted(lis,reverse = True)
    self.eigenvalues = np.array([i[0] for i in lis])
    self.eigvects = np.array([i[1] for i in lis]).T       
    P = np.array(X_)
    if self.n is None:                               # If n = None there will be no change in no. of components.
        self.principal_components = P@(self.eigvects)
    else:
        self.principal_components = P@(self.eigvects[:,:self.n])

    return self.principal_components

  def transform(self,X_):
    mat = self.covmat(X_)
    principal_comp = self.fit(X_)
    return principal_comp

#**MLP**

In [ ]:
class MLP:
      def __init__(self, input_size, hidden_size, output_size, l_rate, func, method):
          self.input_size = input_size
          self.hidden_size = hidden_size
          self.output_size = output_size
          self.l_rate = l_rate
          self.func = func
          self.method = method
      
          if self.method == 'random':
              self.w1 = (np.random.randn(self.input_size, self.hidden_size))
              self.b1 = np.zeros((1, self.hidden_size))
              self.w2 = (np.random.randn(self.hidden_size, self.output_size))
              self.b2 = np.zeros((1, self.output_size))
          elif self.method == 'zero':
              self.w1 = np.zeros((self.input_size, self.hidden_size))
              self.b1 = np.zeros((1, self.hidden_size))
              self.w2 = np.zeros((self.hidden_size, self.output_size))
              self.b2 = np.zeros((1, self.output_size))
          elif self.method == 'constant':
              self.w1 = np.full((self.input_size, self.hidden_size), 0.1)
              self.b1 = np.full((1, self.hidden_size), 0.1)
              self.w2 = np.full((self.hidden_size, self.output_size), 0.1)
              self.b2 = np.full((1, self.output_size), 0.1)

       def activation_func(self, X):
        if self.func == "sigmoid":
            return self.sigmoid(X)
        elif self.func == "relu":
            return self.relu(X)
        elif self.func == "tanh":
            return self.tanh(X)

      def activation_func_derivative(self, X):
        if self.func == "sigmoid":
            return self.sigmoid_derivative(X)
        elif self.func == "relu":
            return self.relu_derivative(X)
        elif self.func == "tanh":
            return self.tanh_derivative(X)

      def sigmoid(self,x):
          return 1 / (1 + np.exp(-x))

      def relu(self,x):
          return np.maximum(0, x)

      def tanh(self,x):
          return np.tanh(x)

      def relu_derivative(self,x):
          return np.where(x > 0, 1, 0)

      def tanh_derivative(self,x):
          return 1 - np.tanh(x)**2

      def sigmoid_derivative(self,x):
          return x * (1 - x)

      def forward(self, X):
          self.hidden_layer = self.activation_func(np.dot(X, self.w1) + self.b1 )
          self.output_layer = self.sigmoid(np.dot(self.hidden_layer, self.w2) + self.b2)

      def backward(self, X, y):
          output_error = y - self.output_layer[0]
          output_delta = (np.array(output_error * self.sigmoid_derivative(self.output_layer[0]))).reshape(-1,1)
          hidden_error = output_delta.dot(self.w2.T)
          hidden_delta = hidden_error * self.activation_func_derivative(self.hidden_layer)

          self.w2 += self.hidden_layer.T.dot(output_delta) * self.l_rate
          self.w1 += X.T.dot(hidden_delta) * self.l_rate

      def train(self, X, y, n_e):
          for e in range(n_e):
              self.forward(X)
              self.backward(X, y)

      def predict(self, X):
          self.forward(X)
          return np.round(self.output_layer)